# Lecture 1: Tools for Scientific Computing 1 (1.5 Hours) #

### ABSTRACT ###

In Lecture 0, we covered the use of the command line, and we installed a variety of packages including SSH and Git (without telling you exactly what they were for). Here comes the punchline: SSH is a cryptographic protocol for communicating over an unsecured network, and Git is software for version control and collaboration. In the first part of this Lecture, we'll show you how to use SSH for secure, remote computing. Then, we'll use SSH together with Git to show how we can really make our lives simpler.

**TODO**:

- Note that ``ssh`` will prompt for host public keys when you try to log in for the first time.

## SSH (Secure Shell) for Remote Computing (40 Minutes) ##

We can use the SSH protocol to run command lines on a remote machine. This is useful for a range of different tasks, but is perhaps most commonly used in scientific computing to run computations on high-performance computing (HPC) resources such as clusters.

Let's start by running a new command-line session, as described in Lecture 0. We've created temporary user accounts on an example remote server, ``epqis.cgranade.com``, so that we have something to SSH into. In the new command-line session (either PowerShell or bash), run the ``ssh`` command below, replacing ``<user>`` by the username provided on your slip of paper:

```bash
$ ssh <user>@epqis.cgranade.com
```

You will then be prompted for the password associated with this username. Type it in and press **enter** to begin your SSH session on the remote machine ``epqis.cgranade.com``. The new SSH session will be heavily encrypted, as is critical for security.

*NB: the ``ssh`` command will not echo your password to you, so the password prompt will look blank. On <i class="fa fa-apple" aria-hidden="true"></i> macOS/OS X, Terminal.app may or may not place a 🔑 emoji in the command line to indicate that echoing is turned off, depending on your version of <i class="fa fa-apple" aria-hidden="true"></i> macOS/OS X.*

Were this an actual useful server, and not (as is actually the case) a Raspberry Pi running in a sweet apartment somewhere, you could then run HPC applications from the comfort of your laptop. Sadly, reality is a little more boring at the moment, so instead feel free to pretend by running a few commands of your choice. Just don't set anything on fire, cause problems with law enforcement, or otherwise act in a very rude fashion with this example server.

Once you're satisfied, run ``exit`` or press **Ctrl-D** to exit the SSH session and return to your own computer's command line. Though this was a useful exercise, it's somewhat limited by one critical problem: we needed to type in our password. This will become very annoying if we have to use SSH as part of an automated process, as will be the case when we learn Git later in this lecture. To solve this, we rely on the *public key infrastructure* (PKI) built into SSH.

Very roughly, in PKI, one can create *keypairs* of a matching private and public key. Anyone with the public key can encrypt a message that can only be decrypted with the matching private key. In SSH, this concept is used to provide a much better alternative to passwords. If you have provided an SSH server with your public key, then the server can ask you to decrypt something in order to prove you have the matching private key. Using PKI in this way means that your password does not have to be sent over the network, greatly reducing your attack surface. The security of SSH's PKI can be further enhanced by using a *passphrase* with your private key. Effectively, a private key with a passphrase cannot be used except by someone who knows that passphrase. Critically, this passphrase is **never** sent across the network, but is only used by your local machine to reconstruct the full private key.

With this bit of handwavy theory out of the way, let's jump in by generating an SSH key.

*NB: Please skip these steps if you already have an SSH key.*

The SSH client we installed in Lecture 0 comes with a handy command ``ssh-keygen`` for doing generating keys.

```bash
$ ssh-keygen
```

You will be prompted where to store the new private key. Press **enter** to accept the default of ``~/.ssh/id_rsa``. You will then be asked to choose a passphrase. Pick something approximately as long as an English sentence, and press **enter** to confirm it. You'll be asked to type it again to help prevent errors. Since this passphrase provides part of the entropy for your private key, it must be quite long compared to traditional passwords. Note that this passphrase cannot be recovered if you forget it— this is by design.

*NB: you should **never** enter your passphrase over a network, as it should **only** be used locally to unlock your private key.*

To tell an SSH server about your *public* key, named ``~/.ssh/id_rsa.pub`` by default, use the ``ssh-copy-id`` command. This will copy the public key to your ``~/.ssh/authorized_keys`` file on the server, which the server then use the next time you try to log in. You should be prompted for your password on the server for the last time in this process.

```bash
$ ssh-copy-id <user>@epqis.cgranade.com
$ ssh <user>@epqis.cgranade.com
```

If all went well, you will instead by prompted by your local machine for the passphrase that you used when generating your key.

A particularly astute and snarky observer may object at this point that we seem to have done something far less convienent than passwords, while justifying our pursuit with convienence as a goal. Indeed, now we need to manage keys *and* type in a far longer string of characters each time we wish to use SSH to do, well, anything. Thankfully, we're not done yet, as we have yet to use an SSH *agent*. An agent is a piece of software running on our local machine that manages SSH keys on our behalf, such that once a key is unlocked by our passphrase, we need not use that passphrase again until the agent decides to lock the key based on its security policy. On <i class="fa fa-apple" aria-hidden="true"></i> macOS/OS X, Keychain acts as an SSH agent and is built into the operating system, such that we should not need to do any further work. Similarly, on <i class="fa fa-linux" aria-hidden="true"></i> Ubuntu, an SSH agent called ``ssh-agent`` is provided by default when we install ``ssh`` using ``apt-get``, as in Lecture 0.

On <i class="fa fa-windows" aria-hidden="true"></i> Windows, the story is a little more complicated, but thankfully we installed everything we need back in Lecture 0. In particular, the problem we run into is that the port of OpenSSH to <i class="fa fa-windows" aria-hidden="true"></i> Windows is in progress, such that even though it supports ``ssh-agent``, this support is not in a form that most <i class="fa fa-windows" aria-hidden="true"></i> Windows programs can make use of. Thus, we will instead use a command-line program ``plink.exe`` that comes with the PuTTY SSH client. This program uses Pageant, the Windows-style SSH agent provided with PuTTY, rather than ``ssh-agent``.

**TODO**:

- Add Pageant to startup menu
- Convert private key to PuTTY format
- Add private key to Pageant
- Run plink
- Add ``GIT_SSH``

- SSH forwarding?

## Version Control with Git (50 Minutes) ##

*NB: We also highly recommend the [tutorial provided by GitHub](https://try.github.io).*

Version-control systems, roughly speaking, provide a structured way of managing *changes* to a project over time, and across a set of collaborators. This allows a user to determmine who changed what and when, to undo changes, and to share changes with collaborators. Many version-control systems, such as Subversion or CVS, are *centralized* in that all changes are uploaded to, downloaded from, and tracked by a single server. This imposes a lot of restrictions however (try using Subversion when you're offline, working on a plane!), so here we'll learn a bit about *decentralized* version control - specifically, we will focus on and learn about the features of Git.

To start off, let's look a bit at how Git stores and manages changes. Effectively, Git is a tool for managing directed acyclic graphs (DAGs), where each node is a set of files that comprises a project (these could be Python scripts, LaTeX files, PNG images, etc). Each edge is then the difference between two sets of files and describes how to reconstruct each node, given the state of a project at some other node.
A graph of this form is called a **repository**, and each node is called a **commit**. 

Probably the easiest way to get a handle on the concepts is to jump right in.

```bash
$ cd ~
$ mkdir epqis16-tmp-repo
$ cd epqis16-tmp-repo
$ git init
Initialized empty Git repository in C:/Users/cgranade/epqis16-tmp-repo/.git/
```

This makes a new, empty *repository* at ``~/epqis16-tmp-repo``. Before anything else, there's one critical command we need to know about: ``git status``. This command gives a brief summary the current status of a repository along with other useful information that we'll explore in more detail later.

```bash
$ git status
On branch master

Initial commit

nothing to commit (create/copy files and use "git add" to track)
```

Importantly, ``git status`` has told us that there's nothing to commit, so let's go on and change that by adding some files.

```bash
$ echo "foobar" > a.txt
$ git status
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)

        a.txt

nothing added to commit but untracked files present (use "git add" to track)
```

Now ``git status`` tells us that there's a file that's *untracked*. This means that it isn't part of any commit, and thus isn't a part of the repository yet. Let's go on and make a new commit to hold ``a.txt``:

```bash
$ git add a.txt
$ git commit -m "Added an important new file."
[master (root-commit) 3fd06ad] Added an important new file.
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 a.txt
$ git status
On branch master
nothing to commit, working tree clean
$ git log
commit 3fd06ad19d9ace8ffd5334bf277d8ee523e60b1a
Author: Chris Granade <cgranade@cgranade.com>
Date:   Thu Oct 27 12:03:45 2016 +1100

    Added an important new file.
```

In the above example, we used the ``-m`` flag (short for "message") to give a short description of the changes in our new commit. Doing so helps out your collaborators (such as future you) figure out what your commit did, so please be polite to yourself and others by writing short but descriptive commit messages.

At any rate, since ``a.txt`` is now in the repository, ``git status`` tells us that there's nothing left to commit at the moment. Suppose, though, that we decide that ``a.txt`` is actually entirely wrong, and need to change it to something else.

```bash
$ echo "baz" > a.txt
$ cat a.txt
baz
$ git status
On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

        modified:   a.txt

no changes added to commit (use "git add" and/or "git commit -a")
```

Now, ``a.txt`` is in the repo, but our changes to it are not. Helpfully, ``git status`` guides us a bit here, and suggests ``git add a.txt`` will do what we need. Let's do that and check ``git status`` one more time.

```bash
$ git add a.txt
$ git status
On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

        modified:   a.txt
```

We can finish the commit as before by running ``git commit``.

```bash
$ git commit -m "Fixed important bug with a.txt."
```

Before moving on, we stress that everything we've done so far has been *offline*. We haven't used any server at *all* to do this, because ``git init`` just made a new repository for us entirely. All of our commits have been to that repository on our local machine. In practice, however, Git really shines when it works with one or more servers (being decentralized, we needn't limit ourselves to just one), as you can then *pull* and *push* changes between your repository and external servers. To do this, then, we can either run our own server (which is not a bad idea, but is well beyond the scope of this workshop), or you can use a service which provides Git hosting for you. Thankfully, several such services exist, including [GitHub](https://github.com/), [Bitbucket](https://bitbucket.org/), [GitLab](https://about.gitlab.com/), and [Visual Studio Team Services](https://www.visualstudio.com/team-services/).

Here, we'll use GitHub as an example, owing to its relative popularity, but most of this example should work on other hosting providers with at most minor modifications. Roughly speaking, we'll proceed in four steps:

- Sign up for an account (and any academic discounts)
- Upload SSH *public* keys to new account
- Make a new repository
- Clone the new reposistory to our laptop

## Setting up a GitHub account ##

**todo (windows-only)**:
- Configuring Git to use SSH with Plink

**todo (OS X and Windows)**:
- Tell Git to use Nano (OS X and Windows only)

**todo (everyone)**:
- if you don't use ``-m`` in ``git commit``, git will open ``$EDITOR``.
- explain how to config merge tool to be VS Code if people like. Otherwise mention Meld, opendiff, etc. as more advanced tools.

The first thing we'll need to do is set up a [GitHub](https://github.com) account. Initially, GitHub allows us to create free *public* repositories, meaning that any files within are publicly available. To avoid trolls pilfering our public repositories for their riches, we'd prefer *private* repositories with restricted access. Fortunately, GitHub provides free private repositories to users with an educational email address - go to [education.github.com](education.github.com), click "Request a discount", enter your GitHub login credentials, complete the brief survery, and await email confirmation. This may take several days, but that's okay because what we create during this workshop will most likely not be monumental enough to keep secret.

In any case, once you have a GitHub account, you can use the web interface to perform many server-side Git operations quickly and easily. Largely, we'll use the GitHub web site to manage things on the server, and command-line Git or Git integration for our preferred text editors to manage things on our local systems. Between the two, we will have a very efficient and flexible version control procedure in place to support our research and other projects.

To do this, though, we'd like to communicate with the GitHub server securely... which is exactly what SSH is for! To assign your SSH public key to your GitHub account, click the *Settings* option from the drop-down User tab in the upper-right corner. In the page that appears, navigate through the *SSH and GPG keys* option, and click on *New SSH key*. In the *Title* box, give your local computer a name. Now we need to give GitHub our public SSH key. Let's display it on the command line. 

```bash
$ cat ~/.ssh/id_rsa.pub
``` 

*NB: Notice the ``.pub``. Without that, you're printing your entire **private** key to the screen.*

Copy the entire public key, beginning with "``ssh-rsa``" and continuing all the way through any web-address-looking characters at the end. Paste it into the *key* field and then *Add SSH key*. 

Now that we've told GitHub our SSH public key, we can use SSH to authenticate ourselves to GitHub in all future steps. Thus, we're now all set to begin using GitHub. Let's make a new repository on GitHub, so that we can sync up our existing ``epqis16-tmp-repo`` to it. Select the *New repository* menu item from the GitHub web interface's "+▼" menu.

<figure style="text-align: center;">
    <img src="files/figures/github-new-repo-menu.png">
    <caption>
        Menu item for creating a new repository on GitHub.
    </caption>
</figure>

This will take you to a new page where you're asked a few things about the new repository you'd like to create.
**todo** 

<figure style="text-align: center;">
    <img src="files/figures/github-new-repo-details.png">
    <caption>
        Filling in the details for a new repository on GitHub.
    </caption>
</figure>

- **Repository name**: Straightforwardly, this is the name of the repository. By default, this is also the name of the folder containing the repository, so something short and easy-to-type is preferable. We suggest lowercase names that are separated by hyphen characters (``-``; if you use an en dash ``–``, you are evil).
- **Description**: A short description that will appear under your repository name on the GitHub page for your new repository. This has no technical impact, so choose something human-readable and informative.
- **Public**/**Private**: Selects whether others can see your repository or not. Unless you have an compelling reason to prefer **Private**, we strongly suggest **Public** for most research uses.
- **Initialize this repository with a README**: For subtle technical reasons, you can't make copies of a Git repository that has no files in it, so GitHub will offer to put at least one file in it to start you off. Since we already have a local repository that we'd like to use, we'll take the advice presented and skip this step for now.
- **Add ``.gitignore``**/**Add a license**: These options allow you to initialize your repository with a ``.gitignore`` file (we'll explain this later) and/or a file describing the copyright license your project falls under. Both of these are almost always good ideas, but we'll skip both for now since we're dealing with a simple example.


Once everything is filled in, press **Create repository** to make your new repository on GitHub. This will take you to a new page that suggests running the following commands in your local repository. Be sure to change the username and repository name based on what you chose above.

```bash
$ git remote add origin git@github.com:cgranade/eqpis16-tmp-repo.git
$ git push -u origin master
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (6/6), 1.73 KiB | 0 bytes/s, done.
Total 6 (delta 0), reused 0 (delta 0)
To github.com:cgranade/eqpis16-tmp-repo.git
 * [new branch]      master -> master
Branch master set up to track remote branch master from origin.
```

We'll break things down a bit more in the next section, but the rough version of what's going on here is that ``git remote add`` tells your local repository to record the existence of another, related repository, and to give that *remote* the name ``origin``. If you create a repository on the server, then copy it to your machine (a process known as *cloning*), then Git will automatically name the server repository ``origin`` for you. **todo: demonstrate how to do that in a separate subsection**

The next command then *pushes* your commits to the server. The ``-u`` flag (short for *upstream*) tells Git which remote that you want to associate with this push. The argument ``master`` refers to a *branch*, but to explain that, we first need a bit more theory.

## Using GitHub for decentralized version control ##

Now that we have a repository that's synced with a server, and have played around with making a few commits of our own, let's take a step back and look at the how everything works in terms of the directed acyclic graph (DAG) structure that we mentioned earlier.

For example, suppose that we've made three commits in total to our new repository. Then, the DAG would have three nodes (one for each commit), each a set of changes from their *parent*, as indicated by a directed edge. The most recent commit in this case is referred to by a kind of label called a *branch*. By default, a new Git repository has only one branch, called ``master``. Putting it all together, the picture looks something like this:

<figure style="text-align: center;">
    <img src="files/figures/git-dag-onlymaster.png" width="60%">
    <caption>
        DAG for a repository containing three commits.
    </caption>
</figure>

Since each remote repository is a complete repository in its own right, our picture has multiple DAGs once we include a remote (note that the GitHub and Bitbucket web interfaces, as well as common desktop applications, will often collapse these into a single picture for brevity; here, we show everything in terms of multiple DAGs to illustrate the concept). For instance, suppose that you make a new commit on your local repository, bringing you to a total of four. In that case, your ``master`` branch moves forward with that commit, but ``origin``'s ``master`` doesn't have that commit and stays in place. Running ``git push`` then makes the new commit available to the server (``origin``) and advances its ``master`` to match our local repository. Pictorially, we represent this with the two DAGs below, with the results of ``git push`` shown in red.

<figure style="text-align: center;">
    <img src="files/figures/git-dag-master-push.png" width="60%">
    <caption>
        DAG for a repository containing four commits, and the results of <span style="color: red;"><tt>git push</tt></span>.
    </caption>
</figure>

The opposite of this, ``git pull``, fetches new commits from remote repositories and advances your branches accordingly. Of course, if this was all Git could do, we'd basically have a rather unwieldy and needlessly complicated reimplementation of Dropbox. Thankfully, where Git really shines is that commits can have multiple children, indicating that different changes are made from the same starting point. We can then merge these later with a special kind of commit called a *merge* (unimaginative naming is sometimes for the best) that has two parents instead of a normal commit, which always has just one parent.

One way that merging can arise is if someone has made changes on the remote that aren't reflected on your local repository and vice versa. Let's add some labels to the commits and see how this might happen:

<figure style="text-align: center;">
    <img src="files/figures/git-dag-premerge.png" width="60%">
    <caption>
        DAG for a repository that will soon need a merge.
    </caption>
</figure>

Here, ``origin`` has a commit ``c`` that we're missing, but we have a commit ``d`` that ``c`` is missing.

*NB: this being a toy example, we've labeled commits as single letters. Normally, commits have names given by cryptographic hashes of their contents, and are often abbreviated to 7-character hexadecimal strings.*

If we try to ``git push`` now, it won't work, because the remote will reject our push and insist that we merge things up locally. Instead, we have to ``git pull``, which will add ``c`` to our local repository and start a new merge that inherits from both ``c`` and ``d``. In many cases, Git can automatically figure out how to make this new merge, because the changes in the two parents won't conflict with each other. (We'll deal with the, er, interesting case below.) After completing the merge, whether automatically or through manual suffering, our local DAG will now look something like this:

<figure style="text-align: center;">
    <img src="files/figures/git-dag-postmerge.png" width="60%">
    <caption>
        Our local repository immediately after merging ``c`` and ``d`` into a new commit ``e``.
    </caption>
</figure>

This now has our ``master`` strictly ahead of ``origin``'s ``master``, so we can ``push`` without losing information. The remote will now accept the push, and we can get on with our lives, knowing we've successfully included both our changes and those from our collaborators (which could include our past selves working on different machines). 

*NB: This also illustrates a common and useful practice of always pulling before pushing. Pulling is, in most ways, much simpler to deal with.*

More commonly, merging happens as a result of introducing new branches. As mentioned before, a branch is just a label for a particular commit. That allows us to attach a label to those commits that indicate work along a particular topic. In their best practices called [GitHub Flow](https://guides.github.com/introduction/flow/), GitHub recomments that whenever you start work on a particular topic within a project, such as new content, a new feature, or a bug fix, you also make a new branch to track that work. Commonly, these topic branches will be given names such as ``feature-foo`` or ``fix-bar``, but neither Git nor GitHub enforce this in any way whatsoever.

Branches are made by the ``git branch`` command, and can then be *checked out* using ``git checkout``. In general, you can checkout a commit to make your local repository reflect the state of the repository at that point in the DAG. Let's take a quick break from the theory to see this in action.

```bash
$ git branch feature-carp
$ git checkout feature-carp
```

This makes a new branch called ``feature-carp``, so named because we'll use it to track a topic that's related to adding carp to our project, then changes our *working copy* to represent this branch. We can confirm this in the usual way with ``git status``:

```bash
$ git status
On branch feature-carp
nothing to commit, working tree clean
```

Let's make a new commit on this branch.

```bash
$ echo "trout" > fish.txt
$ git add fish.txt
$ git commit -m "Needs more fish."
[feature-carp dc99f94] Needs more fish.
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 fish.txt
$ git status
On branch feature-carp
nothing to commit, working tree clean
```

Importantly, we can always go back to another branch, so long as that would not overwrite *uncommited* changes.

```bash
$ git checkout master
$ ls
a.txt
$ git checkout feature-carp
$ ls
a.txt fish.txt
```

If we want to make our new branch available to a remote, we need to say which branch of the remote our new branch should follow. (Some editors, such as VS Code, refer to this as *publishing* a branch, but that terminology is not used by Git itself.) If we try to push without doing this, Git will guide us:

```bash
$ git push
fatal: The current branch feature-carp has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream origin feature-carp
$ git push --set-upstream origin feature-carp
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 941 bytes | 0 bytes/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To github.com:cgranade/eqpis16-tmp-repo.git
 * [new branch]      feature-carp -> feature-carp
Branch feature-carp set up to track remote branch feature-carp from origin.
```

We can now push as normal, since our branch remembers which upstream branch it corresponds to.

```bash
$ echo "carp" > fish.txt
$ git add fish.txt
$ git commit -m "Fixed wrong kind of fish."
[feature-carp 62daacf] Fixed wrong kind of fish.
$ git push
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 943 bytes | 0 bytes/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To github.com:cgranade/eqpis16-tmp-repo.git
   dc99f94..62daacf  feature-carp -> feature-carp
```

Since merging this new branch into ``master`` would be very boring (there's no need for a merge commit, even, as ``master`` has nothing that ``feature-carp`` is missing), let's make the example slightly more interesting by creating a new commit directly on ``master``. This is generally not recommended, but helps us make something quickly.

```bash
$ git checkout master
$ echo "blah" > b.txt
$ git add b.txt
$ git commit -m "Creating conflicts for no reason."
$ git push
```

Going back to the DAG picture, we have something that looks like this:

<figure style="text-align: center;">
    <img src="files/figures/git-dag-premerge-carp.png" width="60%">
    <caption>
        Our local repository immediately before merging ``feature-carp`` into ``master``.
    </caption>
</figure>

We can now merge ``feature-carp`` into ``master`` so that ``master`` reflects both our new changes there and those introduced by the ``feature-carp`` branch.

```bash
$ git merge feature-carp
Merge made by the 'recursive' strategy.
 fish.txt | Bin 0 -> 14 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 fish.txt
$ git push
Counting objects: 2, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 984 bytes | 0 bytes/s, done.
Total 2 (delta 0), reused 0 (delta 0)
To github.com:cgranade/eqpis16-tmp-repo.git
   e4957d8..b016bfe  master -> master
```

Since the ``feature-carp`` branch is now redundant, we can safely delete it without losing any changes.

*NB: this will not delete it on the ``origin`` remote; use the GitHub web interface for that.*

```bash
$ git branch -d feature-carp
Deleted branch feature-carp (was 62daacf).
```

To put it all together, below we've shown a more complicated hypothetical situation involving three branches ``master``, ``feat1`` and ``feat2``, along with the effects of several Git commands. It takes some practice to manage branches and conflicts, but the end result is having a very powerful tool at your disposal for working with your collaborators to get research done.

<figure style="text-align: center;">
    <img src="files/figures/git-dag-finale.jpg" width="60%">
    <caption>
        Our local repository immediately before merging ``feature-carp`` into ``master``.
    </caption>
</figure>

**still todo**

- DAGs
- Edges as diffs, mention latexdiff
- Integration with Sublime Text, VS Code
- Merging in the presence of conflcits
- ``checkout``/``reset`` to manage staging
- make nice pictures